## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os, sys
from pathlib import Path
from ns_extension.wrapper import Splatter, SplatterConfig

import pyvista as pv
pv.start_xvfb()

# import vtkmodules.all as vtk
# render_window = vtk.vtkRenderWindow()
# render_window.SetOffScreenRendering(1)


[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.10.18


[I 07/18/25 17:31:21.097 41561] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/pyvista/plotting/utilities/xvfb.py:48: PyVistaDeprecationWarning: This function is deprecated and will be removed in future version of PyVista. Use vtk-osmesa instead.
  warnings.warn(


Set paths to the file for running splats

In [2]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [6]:
splatter.mesh(
    depth_name="median_depth",
    # sdf_trunc=0.03,
    overwrite=True
)


Available runs:
[0] 2025-07-11_171420


[17:37:56] Auto image downscale factor of 2                                                 ]8;id=661192;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=304477;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[17:38:19] use color only optimization with sigmoid activation                                         ]8;id=247135;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=239330;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

Processing frames:   0%|          | 0/441 [00:00<?, ?it/s]/tmp/ns-extension/ns_extension/utils/camera_utils.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(get_world2view_transform(R, T, trans, scale)).transpose(0, 1).cuda()
Processing frames: 100%|██████████| 441/441 [01:52<00:00,  3.91it/s]


[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=28616
Mapping normals to mesh


Finished computing mesh: 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/mesh/Open3dTSDFfusion.ply

Mapping features to mesh


### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [13]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"],
)

Loading model from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml


[17:46:51] Auto image downscale factor of 2                                                 ]8;id=594417;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=105001;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#484\484]8;;\

[17:47:14] use color only optimization with sigmoid activation                                         ]8;id=206125;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py\splatfacto.py]8;;\:]8;id=417753;file:///opt/conda/envs/nerfstudio/lib/python3.10/site-packages/nerfstudio/models/splatfacto.py#266\266]8;;\

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
/workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/nerfstudio_models/step-00002
9999.ckpt

In [8]:
splatter.plot_mesh()

Number of points: 277426
Number of cells: 494857
Bounds: BoundsTuple(x_min=-1.4198578596115112, x_max=0.7450000047683716, y_min=-0.125, y_max=3.0850000381469727, z_min=-1.459478735923767, z_max=1.81288480758667)


Widget(value='<iframe src="http://localhost:34365/index.html?ui=P_0x73c4ff17c550_3&reconnect=auto" class="pyvi…

Plot similarity maps

In [10]:
splatter.plot_mesh(attribute=similarity)

Number of points: 277426
Number of cells: 494857
Bounds: BoundsTuple(x_min=-1.4198578596115112, x_max=0.7450000047683716, y_min=-0.125, y_max=3.0850000381469727, z_min=-1.459478735923767, z_max=1.81288480758667)


Widget(value='<iframe src="http://localhost:34365/index.html?ui=P_0x73c588277f40_4&reconnect=auto" class="pyvi…

In [14]:
import pyvista as pv

# Load the PLY file
mesh = pv.read(splatter.config['mesh_info']['mesh'])